# EDA Telco Customer Churn

Mục tiêu: hiểu dữ liệu, kiểm tra thiếu/ngoại lệ, phân phối biến và mối liên hệ với `Churn`. Không chạy code trong repo này; dùng notebook này làm sườn khi có Python/venv.

Checklist:
- Kiểm tra kích thước, kiểu dữ liệu, thống kê mô tả.
- Làm sạch `TotalCharges` (chuỗi trống -> NaN -> numeric).
- Phân phối cho biến phân loại và số; boxplot/violin cho outlier.
- Tương quan: heatmap cho số, rate churn theo nhóm cho danh mục.
- Feature drift hoặc imbalance: kiểm tra tỷ lệ churn, SeniorCitizen, Contract.
- Ghi lại phát hiện chính ở cuối notebook.



In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from pathlib import Path

DATA_PATH = Path("../WA_Fn-UseC_-Telco-Customer-Churn.csv")

df = pd.read_csv(DATA_PATH)
print(df.shape)
df.head()


In [ ]:
# Làm sạch TotalCharges
if "TotalCharges" in df.columns:
    df["TotalCharges"] = pd.to_numeric(df["TotalCharges"].replace(" ", np.nan), errors="coerce")

# Tách biến
num_cols = df.select_dtypes(include=["int64", "float64", "int32", "float32"]).columns.tolist()
cat_cols = [c for c in df.columns if c not in num_cols]

# Thiếu dữ liệu
missing = df.isna().mean().sort_values(ascending=False)
missing[missing > 0].head()



In [ ]:
# Phân phối biến số
_ = df[num_cols].hist(bins=30, figsize=(12, 10))
plt.tight_layout()

# Phân phối biến phân loại (top 6 ví dụ)
for col in cat_cols[:6]:
    ax = (df[col].value_counts(normalize=True)
            .sort_values(ascending=False)
            .plot(kind="bar", figsize=(6, 3), title=f"{col} distribution"))
    plt.show()



In [ ]:
# Tỷ lệ churn tổng thể
if "Churn" in df.columns:
    churn_rate = df["Churn"].value_counts(normalize=True)
    print("Churn rate:\n", churn_rate)

# Tương quan giữa biến số
if len(num_cols) > 0:
    plt.figure(figsize=(10, 8))
    corr = df[num_cols].corr()
    sns.heatmap(corr, annot=False, cmap="coolwarm", center=0)
    plt.title("Correlation matrix (numerical)")
    plt.show()

# Tỷ lệ churn theo một số danh mục
for col in [c for c in cat_cols if c != "Churn"][:6]:
    if "Churn" in df.columns:
        rate = df.groupby(col)["Churn"].mean().sort_values(ascending=False)
        ax = rate.plot(kind="bar", figsize=(6, 3), title=f"Churn rate by {col}")
        plt.show()

